In [ ]:
import torch

In [ ]:
! pip install fasttext

     |████████████████████████████████| 68 kB 2.9 MB/s 
  Using cached pybind11-2.8.1-py2.py3-none-any.whl (208 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3121779 sha256=a8b2c63045cbcab59f35ba23547aa05a9fabce3d37ea08133c4be92b8e5d0f04
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
! wget https://www.dropbox.com/s/vttjivmmxw7leea/ru_vectors_v3.bin

import fasttext
ft = fasttext.load_model('ru_vectors_v3.bin')

--2021-12-25 10:26:59--  https://www.dropbox.com/s/vttjivmmxw7leea/ru_vectors_v3.bin
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/vttjivmmxw7leea/ru_vectors_v3.bin [following]
--2021-12-25 10:27:00--  https://www.dropbox.com/s/raw/vttjivmmxw7leea/ru_vectors_v3.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb7a95f77999caa6d595f8a31b4.dl.dropboxusercontent.com/cd/0/inline/BcjLGrAso81G30eVOaxFYGGzBYnz9fvWTAeoCKlsDP-h-MN9Zjj--DLs9yBS7NLNscUgkdJEGkM41SrRSeEClo_FTvlenq3knABM_jgD1w9KtQ3n-Aqhb7yqAH9cUGHdqwKzf2MpdPSufyUJNQZMsWl6/file# [following]
--2021-12-25 10:27:00--  https://ucb7a95f77999caa6d595f8a31b4.dl.dropboxusercontent.com/cd/0/inline/BcjLGrAso81G30eVOaxFYGGzBYnz9fvWTAeoCKlsDP-h-MN9Zjj--DLs9yBS7NLNscUgkdJE

In [ ]:
f = open('ru_pud-ud-test.conllu', 'r')
sent_info = []
sentences, postags, embeddings = [], [], []
for line in f.readlines():
    if len(line) < 2:
        i = 0
        while '# text = ' not in sent_info[i]:
            i += 1
        sent = sent_info[i][9:]
        pos = []
        embs = []
        for j in range(i + 2, len(sent_info)):
            tmp = sent_info[j].split('\t')
            pos.append(tmp[3])
            embs.append(ft.get_word_vector(tmp[1]))
        sentences.append(sent)
        postags.append(pos)
        embeddings.append(embs)
        sent_info = []
    else:
        sent_info.append(line)


In [ ]:
sentences[0], postags[0]

('«Если передача цифровых технологий сегодня в США происходит впервые, то о мирной передаче власти такого не скажешь», – написала Кори Шульман, специальный помощник президента Обамы в своем блоге в понедельник.\n',
 ['PUNCT',
  'SCONJ',
  'NOUN',
  'ADJ',
  'NOUN',
  'ADV',
  'ADP',
  'PROPN',
  'VERB',
  'ADV',
  'PUNCT',
  'SCONJ',
  'ADP',
  'ADJ',
  'NOUN',
  'NOUN',
  'PRON',
  'PART',
  'VERB',
  'PUNCT',
  'PUNCT',
  'PUNCT',
  'VERB',
  'PROPN',
  'PROPN',
  'PUNCT',
  'ADJ',
  'NOUN',
  'NOUN',
  'PROPN',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'PROPN',
  'PUNCT'])

In [ ]:
tags = set()
for elem in postags:
    tags.update(elem)
tag2idx = dict()
idx2tag = ['-'] + list(tags)
for i, t in enumerate(idx2tag):
    tag2idx[t] = i

In [ ]:
postags_idx = [[tag2idx[tag] for tag in sent] for sent in postags]

In [ ]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(embeddings, postags_idx, 
                                                    shuffle=True,
                                                    random_state=42,
                                                    test_size=0.3)

In [ ]:
import numpy as np


def batch_generator(X, y, batchsize, shuffle=True):
    '''
        Генерирует tuple из батча объектов и их меток
    '''
    indices = np.arange(len(X))
    if shuffle:
        indices = np.random.permutation(indices)
    # идем по всем данным с шагом batchsize
    # возвращаем start: start + batchsize объектов на каждой итерации
    for start in range(0, len(indices), batchsize):
        idxs = indices[start:(start + batchsize)]
        X_tmp = [X[i] for i in idxs]
        y_tmp = [y[i] for i in idxs]  
        yield X_tmp, y_tmp

def collate_fn_lm(samples_X, samples_y):
    batch_size = len(samples_X)
    max_len = max(len(sample) for sample in samples_X)
    emb_len = len(samples_X[0][0])

    src_tensor_X = torch.zeros((batch_size, max_len, emb_len), dtype=torch.float)
    src_tensor_y = torch.zeros((batch_size, max_len), dtype=torch.long)

    for batch_id, s in enumerate(samples_y):
        for i, elem in enumerate(s):
            src_tensor_X[batch_id][i][:] = torch.tensor(elem)
    for (batch_id, s) in enumerate(samples_y):
        length = len(s)
        src_tensor_y[batch_id][:length] = torch.tensor(s)

    return src_tensor_X, src_tensor_y

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class BiLSTM_pos_tagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_layers, target_size):
        super(BiLSTM_pos_tagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, bidirectional=True, batch_first=True)
        self.hidden_to_tag = nn.Linear(hidden_dim * 2, target_size)
    
    def forward(self, word_embeddings):
        lstm_out, _ = self.lstm(word_embeddings)
        tag_space = self.hidden_to_tag(lstm_out)
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
! pip install wandb
! wandb login

     |████████████████████████████████| 1.7 MB 4.1 MB/s 
     |████████████████████████████████| 97 kB 7.2 MB/s 
     |████████████████████████████████| 140 kB 43.8 MB/s 
     |████████████████████████████████| 180 kB 49.0 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=939796c24a4cccf349ea5abbcc837881b956de9deeb4ef89968d0316c67c793a
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=6f3dc3f5342626ed0490034bc5475cb40a58570a041f59665d7b0ae9e5c1fb19
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or p

In [ ]:
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
import copy
import wandb
import gc
from tqdm import tqdm
import os

class Trainer:
    def __init__(self, model, 
                 train_X, train_y, val_X, val_y,
                 batch_size=64, lr=1e-4,
                 project="bilstm_pos_tagger", save_path='./'):
        self.loss_func = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        self.train_X = train_X
        self.train_y = train_y
        self.val_X = val_X
        self.val_y = val_y
        self.model = model
        self.save_path = save_path
        self.name = project
        self.batch_size = batch_size
        wandb.init(project=project, entity="imroggen")

    def make_epoch(self, X, y, train=True):
        total_loss = 0
        correct = 0
        total = 0

        if train:
            self.model.train(True)
        else:
            self.model.train(False)
        iters = np.ceil(len(X)/self.batch_size) 
        for X_batch, y_batch in tqdm(
            batch_generator(X, y, batch_size, shuffle=True), 
            total=iters):
            
            X_batch, y_batch = collate_fn_lm(X_batch, y_batch)
            y_batch = torch.LongTensor(y_batch)
            X_batch = torch.FloatTensor(X_batch)
            
            logits = self.model(X_batch.cpu())
            loss = self.loss_func(logits.view(-1, logits.size(-1)), y_batch.view(-1))
            
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            
            mask = (y_batch != 0).to(torch.long)
            pred = torch.argmax(logits, dim=-1)
            correct += ((pred == y_batch)*mask).sum().item()
            total += mask.sum().item()
            total_loss += loss.item()
        return total_loss, correct / total

    @staticmethod
    def log(epoch, train_loss, train_accuracy, test_loss, test_accuracy):
        wandb.log({
            'train': {
                'loss': train_loss,
                'acc': train_accuracy
            },
            'val': {
                'loss': test_loss,
                'acc': test_accuracy
            },
            'epoch': epoch
        })

    def checkpoint(self, epoch):
        torch.save(self.model.state_dict(), os.path.join(self.save_path, self.name + str(epoch) + '.ckpt'))

    def fit(self, max_epochs=20, log=False):
        for epoch in range(max_epochs):
            self.checkpoint(epoch)
            print('\rEpoch: %d' % epoch)
            train_loss, train_acc = self.make_epoch(self.train_X, self.train_y)
            test_loss, test_acc = self.make_epoch(self.val_X, self.val_y, train=False)
            if log:
                self.log(epoch, train_loss, train_acc, test_loss, test_acc)

# Model 1-2-3 (bad quality)

In [ ]:
! mkdir model_1

batch_size = 32
num_layers = 2
hidden_size = 32
embedding_length = ft.get_dimension()
target_size = len(idx2tag)

my_model = BiLSTM_pos_tagger(embedding_length, hidden_size, num_layers, target_size)
trainer1 = Trainer(my_model, train_X, train_y, test_X, test_y, batch_size=batch_size, save_path='./model_1/')
trainer1.fit(log=True)

wandb: Currently logged in as: imroggen (use `wandb login --relogin` to force relogin)


Epoch: 0


100%|██████████| 22/22.0 [00:03<00:00,  7.21it/s]
100%|██████████| 10/10.0 [00:01<00:00,  5.82it/s]


Epoch: 1


100%|██████████| 22/22.0 [00:03<00:00,  6.34it/s]
100%|██████████| 10/10.0 [00:01<00:00,  6.35it/s]


...

100%|██████████| 22/22.0 [00:01<00:00, 15.90it/s]
100%|██████████| 10/10.0 [00:00<00:00, 16.34it/s]


Epoch: 18


100%|██████████| 22/22.0 [00:01<00:00, 15.47it/s]
100%|██████████| 10/10.0 [00:00<00:00, 16.29it/s]


Epoch: 19


100%|██████████| 22/22.0 [00:01<00:00, 16.37it/s]
100%|██████████| 10/10.0 [00:00<00:00, 16.62it/s]


In [ ]:
! mkdir model_2

batch_size = 32
num_layers = 4
hidden_size = 64
embedding_length = ft.get_dimension()
target_size = len(idx2tag)

my_model = BiLSTM_pos_tagger(embedding_length, hidden_size, num_layers, target_size)
trainer1 = Trainer(my_model, train_X, train_y, test_X, test_y, batch_size=batch_size, save_path='./model_2/')
trainer1.fit(log=True)

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch,19


Epoch: 0


100%|██████████| 22/22.0 [00:03<00:00,  5.85it/s]
100%|██████████| 10/10.0 [00:01<00:00,  6.05it/s]


Epoch: 1


100%|██████████| 22/22.0 [00:03<00:00,  6.12it/s]
100%|██████████| 10/10.0 [00:01<00:00,  6.21it/s]


...

100%|██████████| 22/22.0 [00:03<00:00,  5.95it/s]
100%|██████████| 10/10.0 [00:01<00:00,  5.78it/s]


Epoch: 18


100%|██████████| 22/22.0 [00:03<00:00,  5.89it/s]
100%|██████████| 10/10.0 [00:01<00:00,  5.87it/s]


Epoch: 19


100%|██████████| 22/22.0 [00:03<00:00,  5.71it/s]
100%|██████████| 10/10.0 [00:01<00:00,  5.66it/s]


In [ ]:
! mkdir model_3

batch_size = 32
num_layers = 10
hidden_size = 300
embedding_length = ft.get_dimension()
target_size = len(idx2tag)

my_model = BiLSTM_pos_tagger(embedding_length, hidden_size, num_layers, target_size)
trainer1 = Trainer(my_model, train_X, train_y, test_X, test_y, batch_size=batch_size, save_path='./model_3/')
trainer1.fit(log=True)

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch,19


Epoch: 0


100%|██████████| 22/22.0 [01:15<00:00,  3.43s/it]
100%|██████████| 10/10.0 [00:34<00:00,  3.45s/it]


Epoch: 1


100%|██████████| 22/22.0 [01:16<00:00,  3.47s/it]
100%|██████████| 10/10.0 [00:31<00:00,  3.19s/it]


...

100%|██████████| 22/22.0 [01:15<00:00,  3.42s/it]
100%|██████████| 10/10.0 [00:32<00:00,  3.24s/it]


Epoch: 18


100%|██████████| 22/22.0 [01:14<00:00,  3.37s/it]
100%|██████████| 10/10.0 [00:32<00:00,  3.20s/it]


Epoch: 19


100%|██████████| 22/22.0 [01:14<00:00,  3.39s/it]
100%|██████████| 10/10.0 [00:34<00:00,  3.44s/it]


# Model 4-5-6

In [ ]:
class BiLSTM_pos_tagger_v2(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_layers, target_size):
        super(BiLSTM_pos_tagger_v2, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.pca = nn.Linear(embedding_dim, embedding_dim // 2)
        self.lstm = nn.LSTM(embedding_dim // 2, hidden_dim, num_layers, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, hidden_dim)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(hidden_dim, target_size)
    
    def forward(self, word_embeddings):
        pca_emb = self.pca(word_embeddings)
        lstm_out, _ = self.lstm(pca_emb)
        out = self.fc(lstm_out)
        out = self.dropout(out)
        tag_space = self.fc2(out)
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
! mkdir model_4

batch_size = 32
num_layers = 4
hidden_size = 256
embedding_length = ft.get_dimension()
target_size = len(idx2tag)

my_model = BiLSTM_pos_tagger_v2(embedding_length, hidden_size, num_layers, target_size)
trainer1 = Trainer(my_model, train_X, train_y, test_X, test_y, batch_size=batch_size, save_path='./model_4/')
trainer1.fit(log=True)

mkdir: cannot create directory ‘model_4’: File exists


Epoch: 0


100%|██████████| 22/22.0 [00:22<00:00,  1.01s/it]
100%|██████████| 10/10.0 [00:09<00:00,  1.00it/s]


Epoch: 1


100%|██████████| 22/22.0 [00:22<00:00,  1.02s/it]
100%|██████████| 10/10.0 [00:10<00:00,  1.03s/it]


...

100%|██████████| 22/22.0 [00:22<00:00,  1.03s/it]
100%|██████████| 10/10.0 [00:09<00:00,  1.01it/s]


Epoch: 18


100%|██████████| 22/22.0 [00:22<00:00,  1.04s/it]
100%|██████████| 10/10.0 [00:10<00:00,  1.01s/it]


Epoch: 19


100%|██████████| 22/22.0 [00:22<00:00,  1.02s/it]
100%|██████████| 10/10.0 [00:09<00:00,  1.03it/s]


In [ ]:
! mkdir model_5

batch_size = 32
num_layers = 6
hidden_size = 256
embedding_length = ft.get_dimension()
target_size = len(idx2tag)

my_model = BiLSTM_pos_tagger_v2(embedding_length, hidden_size, num_layers, target_size)
trainer1 = Trainer(my_model, train_X, train_y, test_X, test_y, batch_size=batch_size, save_path='./model_5/')
trainer1.fit(log=True, max_epochs=30)

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch,19


Epoch: 0


100%|██████████| 22/22.0 [00:34<00:00,  1.58s/it]
100%|██████████| 10/10.0 [00:15<00:00,  1.55s/it]


Epoch: 1


100%|██████████| 22/22.0 [00:34<00:00,  1.56s/it]
100%|██████████| 10/10.0 [00:15<00:00,  1.56s/it]


...

Epoch: 14


100%|██████████| 22/22.0 [00:34<00:00,  1.56s/it]
100%|██████████| 10/10.0 [00:15<00:00,  1.59s/it]


Epoch: 15


  9%|▉         | 2/22.0 [00:03<00:37,  1.90s/it]


KeyboardInterrupt: ignored

In [ ]:
! mkdir model_6

batch_size = 32
num_layers = 4
hidden_size = 300
embedding_length = ft.get_dimension()
target_size = len(idx2tag)

my_model = BiLSTM_pos_tagger_v2(embedding_length, hidden_size, num_layers, target_size)
trainer1 = Trainer(my_model, train_X, train_y, test_X, test_y, batch_size=batch_size, save_path='./model_6/', lr=1e-4)
trainer1.fit(log=True, max_epochs=30)

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
epoch,14


Epoch: 0


100%|██████████| 22/22.0 [00:30<00:00,  1.37s/it]
100%|██████████| 10/10.0 [00:12<00:00,  1.30s/it]


Epoch: 1


100%|██████████| 22/22.0 [00:29<00:00,  1.35s/it]
100%|██████████| 10/10.0 [00:12<00:00,  1.29s/it]


...

100%|██████████| 22/22.0 [00:29<00:00,  1.34s/it]
100%|██████████| 10/10.0 [00:12<00:00,  1.28s/it]


Epoch: 28


100%|██████████| 22/22.0 [00:29<00:00,  1.34s/it]
100%|██████████| 10/10.0 [00:13<00:00,  1.32s/it]


Epoch: 29


100%|██████████| 22/22.0 [00:29<00:00,  1.35s/it]
100%|██████████| 10/10.0 [00:12<00:00,  1.28s/it]


# Model 7

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
eval_model.load_state_dict(torch.load(args['pos_checkpoint_path']))

In [ ]:
! mkdir model_7

batch_size = 32
num_layers = 4
hidden_size = 500
embedding_length = ft.get_dimension()
target_size = len(idx2tag)

my_model = BiLSTM_pos_tagger_v2(embedding_length, hidden_size, num_layers, target_size)
trainer1 = Trainer(my_model, train_X, train_y, test_X, test_y, batch_size=batch_size, save_path='./model_7/', lr=1e-4)
trainer1.fit(log=True, max_epochs=30)

Epoch: 0


100%|██████████| 22/22.0 [01:17<00:00,  3.50s/it]
100%|██████████| 10/10.0 [00:32<00:00,  3.24s/it]


Epoch: 1


100%|██████████| 22/22.0 [01:16<00:00,  3.49s/it]
100%|██████████| 10/10.0 [00:32<00:00,  3.27s/it]


Epoch: 2


...

100%|██████████| 22/22.0 [01:11<00:00,  3.25s/it]
100%|██████████| 10/10.0 [00:31<00:00,  3.17s/it]


Epoch: 28


100%|██████████| 22/22.0 [01:10<00:00,  3.21s/it]
100%|██████████| 10/10.0 [00:32<00:00,  3.20s/it]


Epoch: 29


100%|██████████| 22/22.0 [01:10<00:00,  3.21s/it]
100%|██████████| 10/10.0 [00:30<00:00,  3.05s/it]
